# "RuntimeError: The size of tensor a (1232) must match the size of tensor b (512) at non-singleton dimension 1" 
Right so BASICALLY this error implies that the maximum input size of the tensor is 512 and the truncation of the dataset said "no truncation limit set" so lets try removing tensor inputs above or equal to 512 and see if that works

In [11]:
from datasets import load_dataset

data set contains train, test and unsupervised data (all with text and labels)

- imdb is a film dataset
- label: 0 means negative review, 1 means positive reviews 
- text: review contents

In [12]:
imdb = load_dataset("imdb")

In [13]:
print(imdb["train"][0:2])

{'text': ['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far b

In [14]:
# for cat in imdb:
#     for input in imdb[cat]:
#         print(len(input['text']))
        

In [15]:
print(imdb)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [16]:
for i in imdb:
    print(i)

train
test
unsupervised


In [17]:
for cat in imdb:
    # L = []
    text = []
    lab = []
    for i in imdb[cat]:
        if len(i['text'].split()) < 512:
            # L.append(i)
            lab.append(i['label'])
            text.append(i['text'])

    print((len(text)*100) / len(imdb[cat]), '%')
    print(len(text), len(imdb[cat]))
    print(i)
    # imdb[cat] = L
    imdb[cat] = {"text": text, "lab" : lab}
    
from datasets import Dataset, DatasetDict
dataset_train = Dataset.from_dict(imdb["train"])
dataset_test = Dataset.from_dict(imdb["test"])
dataset_unsupervised = Dataset.from_dict(imdb["unsupervised"])


dataset_dict = DatasetDict({"train": dataset_train, "test" : dataset_test, "unsupervised" : dataset_unsupervised})


92.368 %
23092 25000
{'text': 'The story centers around Barry McKenzie who must go to England if he wishes to claim his inheritance. Being about the grossest Aussie shearer ever to set foot outside this great Nation of ours there is something of a culture clash and much fun and games ensue. The songs of Barry McKenzie(Barry Crocker) are highlights.', 'label': 1}
92.8 %
23200 25000
{'text': 'I caught this movie on the Sci-Fi channel recently. It actually turned out to be pretty decent as far as B-list horror/suspense films go. Two guys (one naive and one loud mouthed a**) take a road trip to stop a wedding but have the worst possible luck when a maniac in a freaky, make-shift tank/truck hybrid decides to play cat-and-mouse with them. Things are further complicated when they pick up a ridiculously whorish hitchhiker. What makes this film unique is that the combination of comedy and terror actually work in this movie, unlike so many others. The two guys are likable enough and there are so

- Load in a bert model tokenizer
- This distilled model is smaller and faster than a normal bert model but produces the same probabilities
- Built for masking
- Uncased (so England and england are treated as the same work)

In [19]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert/distilbert-base-uncased', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

Function to tokenize text and truncate text to fit bert's maximum input parameter length

In [20]:
def preprocess_function(examples):
    return tokenizer(examples["text"])

- Use map to apply our function
- Batched means it's faster as multiple batches of elements can be processed simultaneously

In [21]:
tokenized_imdb = dataset_dict.map(preprocess_function, batched=True)

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46130/46130 [00:31<00:00, 1477.25 examples/s]


In [22]:
tokenized_imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'lab', 'input_ids', 'attention_mask'],
        num_rows: 23092
    })
    test: Dataset({
        features: ['text', 'lab', 'input_ids', 'attention_mask'],
        num_rows: 23200
    })
    unsupervised: Dataset({
        features: ['text', 'lab', 'input_ids', 'attention_mask'],
        num_rows: 46130
    })
})

- Import batch of examples
- Chose to go with tensorflow for this part (need to add "tf" to return tensor)

In [23]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

2024-05-01 16:25:37.465614: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-01 16:25:39.534542: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [24]:
data_collator

DataCollatorWithPadding(tokenizer=DistilBertTokenizerFast(name_or_path='distilbert/distilbert-base-uncased', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_mul

In [25]:
import evaluate
accuracy = evaluate.load("accuracy")

In [26]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

Map what my labels mean

In [27]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

Create hyperparam variables

In [28]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_imdb["train"]) // batch_size  # a // b seems to be the same as round(a / b, 0)
print(batches_per_epoch, len(tokenized_imdb["train"])/16)
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)
print(optimizer, schedule)

1443 1443.25
<tf_keras.src.optimizers.adam.Adam object at 0x7f987ea2a600> <tf_keras.src.optimizers.schedules.learning_rate_schedule.PolynomialDecay object at 0x7f98ec9602c0>


2024-05-01 16:25:55.405226: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Load: 
- Model
- Number of output labels
- How those labels are mapped

In [29]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label={0: "NEGATIVE", 1: "POSITIVE"}, label2id={"NEGATIVE": 0, "POSITIVE": 1}
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Covert dataset to tf data we can use in our tf model

In [30]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_imdb["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_imdb["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [31]:
tf_validation_set

<_PrefetchDataset element_spec={'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}>

Compile the model (ready for deployment) and attach the optimizer, loss function and any metrics we want to attach

In [32]:
import tensorflow as tf

model.compile(optimizer=optimizer, metrics=['accuracy'])  # No loss argument!

In [33]:
# from transformers.keras_callbacks import KerasMetricCallback

# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)


# from transformers.keras_callbacks import PushToHubCallback

# push_to_hub_callback = PushToHubCallback(
#     output_dir="my_awesome_model",
#     tokenizer=tokenizer,
# )

In [34]:
# callbacks = [metric_callback, push_to_hub_callback]

Train model by fitting data to it and defining epochs

In [35]:
tf_train_set

<_PrefetchDataset element_spec={'input_ids': TensorSpec(shape=(16, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, None), dtype=tf.int64, name=None)}>

In [36]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3)#, callbacks=callbacks)

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


TypeError: in user code:

    File "/anaconda/envs/llm_parser/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/anaconda/envs/llm_parser/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1370, in run_step  *
        outputs = model.train_step(data)
    File "/anaconda/envs/llm_parser/lib/python3.12/site-packages/transformers/modeling_tf_utils.py", line 1641, in train_step  *
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/anaconda/envs/llm_parser/lib/python3.12/site-packages/tf_keras/src/optimizers/optimizer.py", line 620, in minimize  *
        grads_and_vars = self.compute_gradients(loss, var_list, tape)
    File "/anaconda/envs/llm_parser/lib/python3.12/site-packages/tf_keras/src/optimizers/optimizer.py", line 280, in compute_gradients  *
        grads = tape.gradient(loss, var_list)

    TypeError: Argument `target` should be a